# Example_1

In [2]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train.reshape(x_train.shape[0], 784)
x_test  = x_test.reshape(x_test.shape[0], 784)

In [4]:
y_train, y_test = y_train.reshape(-1, 1), y_test.reshape(-1, 1)
enc = OneHotEncoder(handle_unknown='ignore').fit(y_train)
y_train = enc.transform(y_train).toarray()
y_test = enc.transform(y_test).toarray()

In [5]:
x = tf.placeholder(tf.float32, [None, 784])

In [8]:
W_relu = tf.Variable(tf.truncated_normal([784, 100], stddev = 0.1))
b_relu = tf.Variable(tf.truncated_normal([100], stddev = 0.1))

In [9]:
# hudden layer relu
h = tf.nn.relu(tf.matmul(x, W_relu) + b_relu)

In [12]:
# drop_out_layer
keep_probability = tf.placeholder(tf.float32)
h_drop = tf.nn.dropout(h, keep_probability)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [14]:
W = tf.Variable(tf.zeros([100, 10]))
b = tf.Variable(tf.zeros([10]))
# model
y = tf.nn.softmax(tf.matmul(h_drop, W) + b)

In [17]:
# loss
# y_ - y real
y_ = tf.placeholder(tf.float32, [None, 10])
cross_entroty = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
# optimizer
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entroty)
# initialization
init = tf.global_variables_initializer()

In [ ]:
sess = tf.Session()
sess.run(init)

In [18]:
for i in range(1000):
    ind_batch = np.random.randint(x_train.shape[0], size=(1, 100)).flatten()
    batch_xs, batch_ys = x_train[ind_batch], y_train[ind_batch]
    sess.run(train_step, feed_dict={x: batch_xs, y_:batch_ys, keep_probability:1.})

In [20]:
correction_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correction_prediction, tf.float32))

In [22]:
print("Accuracy: %s" %
    sess.run(accuracy, feed_dict={x: x_test, y_: y_test, keep_probability:1.}))

Accuracy: 0.9642


## modification

In [23]:
logit = tf.matmul(h_drop, W) + b

In [26]:
cross_entroty = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logit, labels=y_))

In [27]:
for i in range(10000):
    ind_batch = np.random.randint(x_train.shape[0], size=(1, 100)).flatten()
    batch_xs, batch_ys = x_train[ind_batch], y_train[ind_batch]
    sess.run(train_step, feed_dict={x: batch_xs, y_:batch_ys, keep_probability:1.})

In [28]:
print("Accuracy: %s" %
    sess.run(accuracy, feed_dict={x: x_test, y_: y_test, keep_probability:1.}))

Accuracy: 0.9786
